## Model DistilBert for russian

In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from torch.optim import AdamW
import numpy as np
import torch
import torch.nn as nn
import os
model_name = 'Geotrend/distilbert-base-ru-cased'  # или 'DeepPavlov/rubert-base-cased' для русского
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertForSequenceClassification.from_pretrained(
    model_name,
    num_labels=1  # Один выход для непрерывного значения
)

## Create a DataSet class

In [ ]:
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer
import pandas as pd

class JokesRegressionDataset(Dataset):
    def __init__(self, jokes, scores, tokenizer, max_len):
        self.jokes = jokes
        self.scores = torch.FloatTensor(scores)
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.jokes)

    def __getitem__(self, idx):
        joke = str(self.jokes[idx])
        score = self.scores[idx]

        encoding = self.tokenizer.encode_plus(
            joke,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'rate': score
        }

In [ ]:
df = pd.read_csv(os.path.join('data', 'parsed','data_jokes.csv'))
df.head(5)

In [ ]:
jokes = df['joke'].values
labels = df['rate'].values
print(labels[0:5])
print(jokes[0:5])

In [ ]:
import numpy as np
from sklearn.preprocessing import QuantileTransformer, MinMaxScaler

new_labels = []
new_jokes = []
for j, l in zip(jokes, labels):
    if (l < 1500):
        new_labels.append(l)
        new_jokes.append(j)

new_labels = np.array(new_labels)
new_jokes  = np.array(new_jokes)

def z_normalize(data):
    mean = np.mean(data)
    std_dev = np.std(data)
    return (data - mean) / std_dev
    
# Нормализуем данные
labels_norm = z_normalize(new_labels)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def gr(data):
  # Построим гистограмму распределения
  plt.figure(figsize=(12, 6))

  # Гистограмма с ядерной оценкой плотности
  sns.histplot(data, bins=50, kde=True, color='skyblue')

  # Настройки графика
  plt.title('Распределение оценок смешнявости шуток', fontsize=16)
  plt.xlabel('Оценка смешнявости', fontsize=12)
  plt.ylabel('Количество шуток', fontsize=12)
  plt.grid(axis='y', alpha=0.3)

  # Вертикальные линии для среднего и медианы
  mean_score = data.mean()
  median_score = np.median(data)
  plt.axvline(mean_score, color='red', linestyle='--', label=f'Среднее: {mean_score:.2f}')
  plt.axvline(median_score, color='green', linestyle=':', label=f'Медиана: {median_score:.2f}')

  plt.legend()
  plt.show()
    
gr(labels_norm)

## Separate Dataset to train and validate

In [ ]:
from sklearn.model_selection import train_test_split

# Создание датасетов
MAX_LEN = 128
BATCH_SIZE = 16

# Разделение данных
train_jokes, val_jokes, train_scores, val_scores = train_test_split(
    new_jokes, labels_norm, test_size=0.3, random_state=42
)

# Создание DataLoader'ов
train_dataset = JokesRegressionDataset(train_jokes, train_scores, tokenizer, MAX_LEN)
val_dataset = JokesRegressionDataset(val_jokes, val_scores, tokenizer, MAX_LEN)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

## Train model

In [ ]:
from tqdm.auto import tqdm
from torch.amp import autocast, GradScaler
# Используем MSELoss для регрессии
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.cuda.empty_cache()

model = model.to(device)

loss_fn = nn.MSELoss().to(device)
optimizer = AdamW(model.parameters(), lr=2e-5)
scaler = GradScaler(device)

def train_epoch(model, data_loader, loss_fn, optimizer, scaler, device):
    model.train()
    losses = []

    progress_bar = tqdm(data_loader, desc="Training", leave=False)

    for d in progress_bar:
        optimizer.zero_grad()

        input_ids = d['input_ids'].to(device)
        attention_mask = d['attention_mask'].to(device)
        scores = d['rate'].to(device)

        with autocast(dtype=torch.float16, device_type=device):
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )

            preds = outputs.logits.squeeze()
            loss = loss_fn(preds, scores)

        losses.append(loss.item())

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        progress_bar.set_postfix({'loss': f'{np.mean(losses[-10:]):.4f}'})

    return np.mean(losses)

def eval_model(model, data_loader, loss_fn, device):
    model.eval()
    losses = []
    progress_bar = tqdm(data_loader, desc="Validation", leave=False)
    with torch.no_grad():
        for d in progress_bar:
            input_ids = d['input_ids'].to(device)
            attention_mask = d['attention_mask'].to(device)
            scores = d['rate'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )

            preds = outputs.logits.squeeze()
            loss = loss_fn(preds, scores)

            losses.append(loss.item())

            progress_bar.set_postfix({'loss': f'{np.mean(losses[-10:]):.4f}'})

    return np.mean(losses)

# Обучение
EPOCHS = 3
prev_val_loss = np.inf
for epoch in range(EPOCHS):
    print(f'Epoch {epoch + 1}/{EPOCHS}')
    print('-' * 10)

    train_loss = train_epoch(model, train_loader, loss_fn, optimizer, scaler, device)
    print(f'Train loss: {train_loss:.4f}')

    val_loss = eval_model(model, val_loader, loss_fn, device)
    print(f'Validation loss: {val_loss:.4f}\n')

    if (prev_val_loss > val_loss):
        prev_val_loss = val_loss
        
        model.save_pretrained(str(epoch) + '_funny_jokes_classifier.model')
        tokenizer.save_pretrained(str(epoch) + '_funny_jokes_classifier.tokenizer')

## Eval Model

In [ ]:
# Загрузка модели
model = DistilBertForSequenceClassification.from_pretrained('0_funny_jokes_classifier.model')
tokenizer = DistilBertTokenizer.from_pretrained('0_funny_jokes_classifier.tokenizer')

# Предсказание на новом тексте
def predict_funny_score(model, joke):
    model.to('cpu')
    # Токенизация
    encoded_joke = tokenizer.encode_plus(
        joke,
        max_length=MAX_LEN,
        add_special_tokens=True,
        return_token_type_ids=False,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt',
    )

    input_ids = encoded_joke['input_ids'].to('cpu')
    attention_mask = encoded_joke['attention_mask'].to('cpu')

    output = model(input_ids, attention_mask)

    # Получаем сырое предсказание
    prediction = output.logits.squeeze()

    return torch.sigmoid(prediction)

# Пример использования
joke_examples = [
    "Почему программисты путают Хэллоуин и Рождество? Потому что Oct 31 == Dec 25!",
    "Как объяснить бабушке, что такое баг? Вот смотри: включи телевизор. Не включается? Вот это баг!",
    "Очень плохая шутка, вообще не смешная",
    "Скелет заходит в бар и заказывает пиво и швабру.",
    "Штирлиц три года прожил в браке с немкой. Ни разу не заподозрил, что она женщина.",
    "Штирлиц стрелял вслепую. Слепая умерла.",
    "— Доктор, я буду жить? — А смысл?",
    "Штирлиц ехал на машине и увидел знак «Осторожно, мины!». «Ну вот, опять адрес стерли», — подумал он.",
    "Перестаньте учить своих детей, что война – это слава и героизм. Научите их, что настоящая слава – в предотвращении войн, и что герои – это те, кто может это сделать.",
    "Идёт суд над акушером. Обвиняемый: - Ваша честь, если бы вы только увидели этого младенца, вы бы его тоже обратно затолкали!",
    "Деньги на «Чёрный день» лучше хранить в тёмной комнате.",
    "Привет. это разве шутка?",
    "Почему программисты путают Хэллоуин и Рождество? Потому что Oct 31 == Dec 25!",
    "Как объяснить бабушке, что такое баг? Вот смотри: включи телевизор. Не включается? Вот это баг!",
    "Очень плохая шутка, вообще не смешная",
    "Скелет заходит в бар и заказывает пиво и швабру.",
    "Штирлиц три года прожил в браке с немкой. Ни разу не заподозрил, что она женщина.",
    "Штирлиц стрелял вслепую. Слепая умерла.",
    "— Доктор, я буду жить? — А смысл?",
    "Штирлиц ехал на машине и увидел знак «Осторожно, мины!». «Ну вот, опять адрес стерли», — подумал он.",
    "— Как отличить трудоголика от самоубийцы? — Трудоголик прыгает с крыши в рабочее время.",
    "— Почему утонувший мальчик не смог позвать на помощь? — Потому что он был вежливым и не перебивал воду.",
    "А разве первыми в военкомат вызывают не тех, у кого на машине наклеено: \"Можем повторить!\"?",
    "Мама Димы больше не звонит и не спрашивает «поел ли сынок». Она просто подписалась на его инстаграм.",
    "Наркоман Павлик долго боялся урны со своей кремированной бабушкой, потом ничего, ВТЯНУЛСЯ!"
    "Заходит бесконечное количество математиков в бар Мужики, вы не понимаете?",
    "Заходит бесконечное количество математиков в бар Ну что ж, они идут к пиву, а там стойка: - Слушайте, вы знаете как решить систему уравнений? - Да. - А какой способ? - Ну... - Ну, вы мне не расскажите? - Да. - Хорошо, тогда давайте начнем с первого уравнения. И так далее...",
    "Заходит бесконечное количество математиков в бар Из них вылезает один и говорит: - Девчонки, я вам скажу что такое число 1. Мужики смотрят на него с надеждой. - Да, вы знаете? - Конечно! Это число, которое когда вы его делите на себя, то получается 0! - И это же не может быть числом, потому что 1/1=1, а мы уже знаем, что это число 1.",
    "Заходит бесконечное количество математиков в бар - Вы, господа, не знаете, сколько пива я выпью сегодня?"
    "Заходит бесконечное количество математиков в бар Они там сидят и ужинают, а потом выходят.",
    "Заходит бесконечное количество математиков в бар Все они спрашивают у хозяина: - Вы что, не видите? Моя жена мне говорит, что я ей на фига делаю! Хозяин смотрит на них и отвечает: - Ну да, ну да... Я же вам сказал - вы все такие же!"
]

for joke in joke_examples:
    score = predict_funny_score(model, joke)
    print(f'Шутка: "{joke[:50]}..."\nОценка смешнявости: {score:.2f}\n')